# Worksheet 05

Name:  Grace Elias
UID: U91459237

### Topics

- Cost Functions
- Kmeans

### Cost Function

Solving Data Science problems often starts by defining a metric with which to evaluate solutions were you able to find some. This metric is called a cost function. Data Science then backtracks and tries to find a process / algorithm to find solutions that can optimize for that cost function.

For example suppose you are asked to cluster three points A, B, C into two non-empty clusters. If someone gave you the solution `{A, B}, {C}`, how would you evaluate that this is a good solution?

Notice that because the clusters need to be non-empty and all points must be assigned to a cluster, it must be that two of the three points will be together in one cluster and the third will be alone in the other cluster.

In the above solution, if A and B are closer than A and C, and B and C, then this is a good solution. The smaller the distance between the two points in the same cluster (here A and B), the better the solution. So we can define our cost function to be that distance (between A and B here)!

The algorithm / process would involve clustering together the two closest points and put the third in its own cluster. This process optimizes for that cost function because no other pair of points could have a lower distance (although it could equal it).

### K means

a) (1-dimensional clustering) Walk through Lloyd's algorithm step by step on the following dataset:

`[0, .5, 1.5, 2, 6, 6.5, 7]` (note: each of these are 1-dimensional data points)

Given the initial centroids:

`[0, 2]`

[0, .5] [1.5, 2, 6, 6.5, 7]
centroids: [.25, 4.6]
[0, .5, 1.5, 2] [6, 6.5, 7]
centroids: [1, 6.5]
[0, .5, 1.5, 2] [6, 6.5, 7]

b) Describe in plain english what the cost function for k means is.

The cost function for k means looks to find the optimal centroids for k clusters so that all clusters have minimal variance.

c) For the same number of clusters K, why could there be very different solutions to the K means algorithm on a given dataset?

The randomness of the initial K centroids will affect how the clusters form. It is possible to find a local optimum instead of a global optimum.

d) Does Lloyd's Algorithm always converge? Why / why not?

Lloyd's Algorithm always converges, but it may converge to a local optimum rather than the global optimum.

e) Follow along in class the implementation of Kmeans

In [2]:
pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 2.4 MB/s eta 0:00:0000:010:01
You should consider upgrading via the '/Users/graceelias/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import sklearn.datasets as datasets

centers = [[0, 0], [2, 2], [-3, 2], [2, -4]]
X, _ = datasets.make_blobs(n_samples=300, centers=centers, cluster_std=1, random_state=0)

class KMeans():

    def __init__(self, data, k):
        self.data = data
        self.k = k
        self.assignment = [-1 for _ in range(len(data))]
        self.snaps = []
    
    def snap(self, centers):
        TEMPFILE = "temp.png"

        fig, ax = plt.subplots()
        ax.scatter(X[:, 0], X[:, 1], c=self.assignment)
        ax.scatter(centers[:,0], centers[:, 1], c='r')
        fig.savefig(TEMPFILE)
        plt.close()
        self.snaps.append(im.fromarray(np.asarray(im.open(TEMPFILE))))
        
    def is_unassigned(self, i):
        return self.assignment[i] == -1
    
    def unassign_all(self):
        self.assignment = [-1 for _ in range(len(self.data))]
        
    def initialize(self):
        return self.data[np.random.choice(range(len(self.data)), size=self.k, replace=False)]
    
    def are_centers_diff(self, c1, c2):
        for i in range(len(c1)):
            if c1[i] not in c2:
                return True
        return False
    
    def assign(self, centers):
        for i in range(len(self.data)):
            self.assignment[i] = 0
            temp_assign = 0
            temp_dist = self.dist(self.data[i], centers[0])
            for j in range(1, len(centers)):
                new_dist = self.dist(self.data[i], centers[j])
                if temp_dist > new_dist:
                    self.assignment[i] = j
                    temp_dist = new_dist
    
    def calculate_new_centers(self):
        centers = []
        for j in range(self.k):
            cluster_j = self.data[
                np.array([i for i in range(len(self.data)) if self.assignment[i] == j])
            ]
            centers.append(np.mean(cluster_j,axis=0))
        
        return np.array(centers)

    def dist(self, x, y):
        return sum((x - y) ** 2) ** (1/2)

    def lloyds(self):
        centers = self.initialize()
        self.assign(centers)
        self.snap(centers)
        new_centers = self.calculate_new_centers()
        while self.are_centers_diff(centers, new_centers):
            centers = new_centers
            self.snap(centers)
            self.unassign_all()
            self.assign(centers)
            new_centers = self.calculate_new_centers()
            print (new_centers)
        return
            
kmeans = KMeans(X, 4)
kmeans.lloyds()
images = kmeans.snaps

images[0].save(
    'kmeans.gif',
    optimize=False,
    save_all=True,
    append_images=images[1:],
    loop=0,
    duration=500
)

[[-3.8921717   2.07173166]
 [-1.81029647  1.13614988]
 [ 1.64472455 -3.65310842]
 [ 1.53380025  1.5597265 ]]
[[-3.73054442  1.90911046]
 [-1.45436541  0.9412462 ]
 [ 1.74766928 -3.87138518]
 [ 1.58798487  1.51486356]]
[[-3.63315964  1.92123563]
 [-1.17423975  0.68743589]
 [ 1.81549664 -3.96557375]
 [ 1.68506495  1.62313433]]
[[-3.30438614  1.90372314]
 [-0.69867573  0.35412888]
 [ 1.83280221 -3.99671901]
 [ 1.78389699  1.69899856]]
[[-3.15636271  1.8740792 ]
 [-0.35015136  0.15409155]
 [ 1.84180743 -4.03258775]
 [ 1.86471673  1.81986805]]
[[-3.10208548  1.85147285]
 [-0.16263312  0.08983788]
 [ 1.84180743 -4.03258775]
 [ 1.90165131  1.93255625]]
[[-3.07703402  1.85266958]
 [-0.03760357  0.01175879]
 [ 1.84585117 -4.10098602]
 [ 1.904406    1.98450725]]
[[-3.07703402  1.85266958]
 [-0.01886062  0.02039989]
 [ 1.84585117 -4.10098602]
 [ 1.91248297  2.00204407]]
[[-3.07703402  1.85266958]
 [-0.01886062  0.02039989]
 [ 1.84585117 -4.10098602]
 [ 1.91248297  2.00204407]]
